## 4.) Write a program to compute unsmoothed unigrams and bigrams.
### a.) Use any language you want, but do not use any libraries other than math/probability ones (Java.Math, numpy, etc
    - This means you cannot use SpaCy or NLTK
### b.) Run your n-gram program on two different small corpora of your choice. Try and make them from different genres (i.e. a news article and a song lyric)
    - Unigrams is just the list of tokens
    - Bigrams is the list of two token sequences
### c.) Now compare the statistics of the two corpora, what are the differences in the most common unigrams between the two? How about interesting differences in bigrams?
    - Going to need to compute the counts of which bigram and which unigram is most common
### d.) Add an option to your program to generate random sentences.
    - Going to need to compute probabilities and add a variance param so that we do not get the same sentence over and over

In [1]:
from pathlib import Path
from collections import Counter
import pandas as pd
import random
from tqdm import tqdm

In [2]:
file1 = "The_Boy_Travellers_in_the_Far_East_Part_Third_by_Thomas_Wallace_Knox.txt"
file2 = "The_Cathedrals_of_Great_Britain_Their_History_and_Architecture_by_PH_Ditchfield.txt"

In [3]:
# # method to check what bigrams look like
# # https://stackoverflow.com/a/32442106
# import nltk
# from nltk import ngrams
# text1 = Path(file1).read_text()
# token = nltk.word_tokenize(text1)
# bigrams = ngrams(token, 2)
# frequencies1 = Counter(bigrams)
# frequencies1

In [4]:
# https://stackoverflow.com/a/31505798
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
# prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
### ADDED IN FROM COMMENTS
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
### END
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    ### ADDED IN FROM COMMENTS
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "e.g." in text: text = text.replace("e.g.","e<prd>g<prd>")
    if "i.e." in text: text = text.replace("i.e.","i<prd>e<prd>")
    ### END
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [5]:
start_of_sent_token = "<S>"
end_of_sent_token = "</S>"

def compute_ngram(filename: str):
    text = Path(filename).read_text()
    sents = split_into_sentences(text)

    tokens = []
    for sent in sents:
        sent_tokens = list(sent.split())
        sent_tokens.insert(0, start_of_sent_token)
        sent_tokens.insert(len(sent_tokens), end_of_sent_token)
        tokens += sent_tokens
    unigrams = Counter(tokens).most_common()

    # probability = n occurances of a given word / total number of all words in the data (not unique)
    total_tokens = len(tokens)
    # unigram_df = pd.DataFrame(columns=["unigram", "count", "probability"])  # slow
    unigram_probabilities = {}
    unigram_counts = {}
    unigram_list = []
    for unigram, count in unigrams:
        p = count / total_tokens
        # unigram_df = pd.concat([unigram_df, pd.DataFrame.from_records([{"unigram": unigram, "count": count, "probability": p}])])  # slow
        unigram_probabilities[unigram] = p
        unigram_counts[unigram] = count
        unigram_list += [(unigram, count, p)]
    unigram_df = pd.DataFrame(unigram_list, columns=['unigram', 'count', 'probability'])

    bigram_tokens = []
    for idx in range(1, len(tokens), 1):
        given = tokens[idx - 1]
        word = tokens[idx]
        bigram = (given, word)
        bigram_tokens.append(bigram)
    bigrams = Counter(bigram_tokens).most_common()

    # bigram_probabilities = []
    bigram_list = []
    for bigram, count in bigrams:
        given, token = bigram
        count_given_unigram = unigram_counts[given]
        # count_given_unigram = unigram_df[unigram_df['unigram'] == bigram[0]]['count']  # slow
        probability = count / count_given_unigram
        # bigram_probabilities += [(bigram, probability)]
        bigram_list += [(bigram, token, given, count, probability)]


    bigram_df = pd.DataFrame(bigram_list, columns=['bigram', 'token', 'given', 'count', 'probability'])
    # print(unigram_df.head())
    return unigram_df, bigram_df

In [6]:
unigram_df1, bigram_df1 = compute_ngram(file1)
unigram_df2, bigram_df2 = compute_ngram(file2)

In [7]:
unigram_df1

,unigram,count,probability
0,the,11060,0.072095
1,<S>,5835,0.038036
2,</S>,5835,0.038036
3,of,5736,0.037390
4,and,5560,0.036243
...,...,...,...
18422,"Gutenberg-tm,",1,0.000007
18423,subscribe,1,0.000007
18424,email,1,0.000007
18425,newsletter,1,0.000007


In [8]:
unigram_df2

,unigram,count,probability
0,the,12098,0.080331
1,<S>,7938,0.052709
2,</S>,7938,0.052709
3,of,7393,0.049090
4,and,5237,0.034774
...,...,...,...
18438,facility:,1,0.000007
18439,"Gutenberg-tm,",1,0.000007
18440,subscribe,1,0.000007
18441,email,1,0.000007


In [9]:
bigram_df1

,bigram,token,given,count,probability
0,"(</S>, <S>)",<S>,</S>,5834,0.999829
1,"(of, the)",the,of,2114,0.368550
2,"(<S>, The)",The,<S>,900,0.154242
3,"(in, the)",the,in,803,0.310638
4,"(and, the)",the,and,667,0.119964
...,...,...,...,...,...
74546,"(newsletter, to)",to,newsletter,1,1.000000
74547,"(hear, about)",about,hear,1,0.055556
74548,"(about, new)",new,about,1,0.002833
74549,"(new, eBooks.)",eBooks.,new,1,0.031250


In [10]:
bigram_df2

,bigram,token,given,count,probability
0,"(</S>, <S>)",<S>,</S>,7937,0.999874
1,"(of, the)",the,of,2759,0.373191
2,"(<S>, The)",The,<S>,2171,0.273495
3,"(in, the)",the,in,990,0.396158
4,"(and, the)",the,and,662,0.126408
...,...,...,...,...,...
67318,"(email, newsletter)",newsletter,email,1,1.000000
67319,"(newsletter, to)",to,newsletter,1,1.000000
67320,"(hear, about)",about,hear,1,0.166667
67321,"(about, new)",new,about,1,0.009615


In [11]:
def generate_sentence(bigram_df: pd.DataFrame, sentence=[start_of_sent_token], variance_amount=10, max_words=20) -> list:
    token_to_find = sentence[-1]
    # print(len(sentence) < max_words)
    if token_to_find != end_of_sent_token and len(sentence) < max_words:
        bigram_matches = bigram_df[bigram_df['given'] == token_to_find]\
            .sort_values(by=['probability'], ascending=False)\
            .reset_index(drop=True)\
            .head(variance_amount + 1)
        if len(bigram_matches['bigram']) >= 0:
            # pick random one in top variance amount
            next_token_idx = random.randint(0, min(variance_amount, len(bigram_matches['bigram']) - 1))  # both inclusive
            try:
                sentence += [str(bigram_matches.iloc[next_token_idx]['token'])]
            except IndexError:
                print(len(bigram_matches['bigram']))
                print(next_token_idx)
                print(bigram_matches)
            # print(sentence)
            return generate_sentence(bigram_df, sentence, variance_amount=variance_amount, max_words=max_words)
    else:
        return sentence

def generate_sentences(bigram_df: pd.DataFrame, n=5, variance_amount=10, max_words=20):
    # sent = generate_sentence(bigram_df, sentence=[start_of_sent_token], variance_amount=variance_amount, max_words=max_words)
    # print(sent)
    for i in range(n):
        print(' '.join(generate_sentence(bigram_df, sentence=[start_of_sent_token], variance_amount=variance_amount, max_words=max_words)))


In [12]:
print(file1)
generate_sentences(bigram_df1, n=5, variance_amount=10, max_words=35)

print()
print(file2)
generate_sentences(bigram_df2, n=5, variance_amount=10, max_words=35)

The_Boy_Travellers_in_the_Far_East_Part_Third_by_Thomas_Wallace_Knox.txt
<S> In each person made a long time at a couple of these animals are very large one place called 'The Black Town,' and they had seen from which it would not more about thirty
<S> The latter country where the boys asked about fifty days they were on his feet from all its course the time at the Doctor to see any of all its own employer. </S>
<S> They saw was a dozen or three other tropical vegetation, among them are in their boats on account would sell it will have not at their way back to be in India to do
<S> A gentleman whose finger-nails had not only in the most famous 'Peacock Throne,' which they would rather to get to the natives make an elephant in any means," was about twenty yards more or
<S> A few hundred days through it. </S>

The_Cathedrals_of_Great_Britain_Their_History_and_Architecture_by_PH_Ditchfield.txt
<S> Here we find a few specimens of St. Mary, wife and are Early Decorated period, 1296 A.D., it 